In [3]:
import pandas as pd
import numpy as np
from fastapi import FastAPI
from fastapi import Request
from fastapi.templating import Jinja2Templates
from flask import Flask, request, jsonify

In [2]:
app = FastAPI()

In [19]:
class SistemaRecomendacion:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def limpiar_nombre(self, nombre):
        # Función para limpiar el nombre del juego
        return nombre.lower().capitalize()

    def recomendacion_juego(self, id_producto:int):
        # Función para recomendar juegos similares
        juego = self.dataframe.loc[self.dataframe['ID'] == id_producto, 'Nombre'].iloc[0]
        juego_limpio = self.limpiar_nombre(juego)
        # Implementar el algoritmo de recomendación item-item aquí
        # Por ahora, devolvemos juegos similares ficticios
        juegos_similares = ['Juego 1', 'Juego 2', 'Juego 3', 'Juego 4', 'Juego 5']
        return juegos_similares

In [ ]:
df_sistRec = pd.read_parquet('venv/data/steam_games_etl_comprimido.parquet')

In [ ]:
# Crear una instancia del sistema de recomendación
sistema_recomendacion = SistemaRecomendacion(df_sistRec)

# Crear una aplicación Flask
app = Flask(__name__)


In [38]:
import pandas as pd
from sklearn.metrics import jaccard_score

class SistemaRecomendacion:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def calcular_similitud(self, juego_referencia):
        # Obtener el género del juego de referencia y convertirlo en lista
        genero_referencia = self.dataframe.loc[self.dataframe['id'] == juego_referencia, 'genres'].iloc[0].split(', ')
        
        # Calcular la similitud de Jaccard entre el género del juego de referencia y todos los demás juegos
        similitudes = {}
        for index, row in self.dataframe.iterrows():
            if row['id'] != juego_referencia:
                genero_otro_juego = row['genres'].split(', ')
                similitud = jaccard_score(genero_referencia, genero_otro_juego, average='macro')
                similitudes[row['app_name']] = similitud

        # Ordenar los juegos por similitud y devolver los 5 más similares
        juegos_similares = sorted(similitudes.items(), key=lambda x: x[1], reverse=True)[:5]
        return juegos_similares


In [39]:
# Crear una instancia del sistema de recomendación con el DataFrame df_sistRec
df_sistRec = pd.read_parquet('venv/data/steam_games_etl_comprimido.parquet')
sistema_recomendacion = SistemaRecomendacion(df_sistRec)

In [41]:
# Obtener recomendaciones para un juego específico
id_producto_consulta = 761140  # Cambia el ID del producto que deseas consultar
recomendaciones = sistema_recomendacion.calcular_similitud(id_producto_consulta)
print("Recomendaciones para el juego con ID", id_producto_consulta, ":")
for juego, similitud in recomendaciones:
    print(juego, "- Similitud:", similitud)

Recomendaciones para el juego con ID 761140 :
Pixel Puzzles 2: Anime - Similitud: 1.0
World of Cinema - Directors Cut - Similitud: 1.0
Toy Soldiers: Complete - Similitud: 1.0
Pixel Puzzles 2: RADical ROACH - Similitud: 1.0
Aerial Destruction - Similitud: 1.0
